# OSM Shapefile Decoding

In [9]:
## Basic stuff
#%load_ext autoreload
#%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install
import shapefile
import geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from fsUtils import isFile
from ioUtils import showSize, saveJoblib
from geoUtils import *
from geospatialUtils import saveGeoData, getBB, rmZipDir, getGeos
import pickle
from glob import glob
from os.path import basename,splitext,join
from collections import Counter

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.5 |Anaconda custom (x86_64)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-11-19 10:14:09.352731


In [1]:
# Global Params
basedir="/Users/tgadf/Downloads/osm"
prec=7
mainkey="OSM"

# Parse OSM Data

In [10]:
from ioUtils import loadJoblib
buildingLookup = loadJoblib("buildingLookup.p")
buildingMap    = loadJoblib("buildingMap.p")
buildingConv   = loadJoblib("buildingConv.p")

fastfood=["fastfood", "foodcourt","convenience", "cafe"]
restaurant=["restaurant"]
homefood=["greengrocer", "pharmacy", "supermarket", "chemist"]
lodging=["hotel", "motel", "hostel","chalet","bedandbreakfast","guesthouse", "caravansite"]
commercial=["bookshop","butcher","hairdresser", "furnitureshop", "doityourself", "gardencentre", "newsagent", "toyshop","mall","laundry","mobilephoneshop","bank","atm"]
commercial+=["bicyclerental","bicycleshop","bakery","beautyshop","videoshop","outdoorshop", 'jeweller','kiosk', 'shoeshop','sportsshop', "travelagent"]
commercial+=['florist','giftshop', "departmentstore", "computershop"]
vending=['vendingany','vendingcigarette','vendingmachine','vendingparking']
commercial+=vending
auto=['cardealership','carrental','carrepair','carsharing','carwash']
municipal=["firestation", "prison", "library", "postoffice", "postbox", "police", "townhall","communitycentre","artscentre", "publicbuilding", "courthouse"]
industrial=["wastewaterplant", "windmill","watermill","waterworks", "recycling", "recyclingclothes", "recyclingglass", "recyclingmetal", "recyclingpaper", "commstower", 'watertower','waterwell', "tower"]
building=["embassy"]
medical=["hospital", "dentist", "doctors", "optician", "veterinary","nursinghome"]
attraction=["attraction","fountain", "museum", "touristinfo", "viewpoint", "zoo","themepark","monument","picnicsite","fort","castle","battlefield","archaeological","memorial", "lighthouse", "observationtower", "artwork"]
religious=["graveyard","waysidecross","waysideshrine","ruins"]
sport=["golfcourse", "huntingstand", "icerink", "sportscentre", "track", "swimmingpool"]
entertainment=["bar", "pub", "nightclub", "theatre", "cinema"]
recreation=["stadium", "campsite","biergarten","dogpark","park", "pitch", "playground"]
school=["kindergarten","school"]
college=["college","university"]
general=['alpinehut','bench','beverages','camerasurveillance','clothes','drinkingwater','shelter','stationery','telephone','toilet','wastebasket']                

increase={}

def parseOSM(dirval, prec, key=""):
    from os.path import basename, dirname, join, splitext
    from glob import glob
    from collections import Counter
    state = basename(dirval).replace("-latest-free.shp", "")
    useUnique = False
    osmpath = dirval
    osms     = [splitext(basename(x))[0] for x in glob(join(osmpath, "*{0}*.shp".format(key)))]
    blgCntr = Counter()
    nameCntr = Counter()
    nameLookup = {}
    
    
    shapeData   = {}
    geoShapeMap = {}
    ngeos     = 0
    totalgeos = 0
    show      = False
    cntr      = Counter()
        
    print("Found the following {0} shape files: {1}".format(len(osms), ", ".join(osms)))
    for osm in osms:
        print(osm)
        shapeval = osm
        
        files = glob(join(basedir, "{0}-{1}-{2}*".format(state, osm, prec)))
        if len(files) > 0:
            print("Already processed this one...")
            #continue

        try:
            sf = shapefile.Reader(join(osmpath, osm))
        except:
            raise ValueError("No shapefile!")
        fields      = sf.fields
        Nshapes   = len(sf.shapes())

        start,cmt = clock("\n\nAnalyzing {0}\t{1}".format(shapeval, Nshapes))
        if show:
            print("Fields -> {0}".format(fields))

        irec = -1
        for shapeRec in sf.iterShapeRecords():
            irec += 1
            if irec % 2500 == 0 and irec > 0:
                if True:
                    print("Processed {0}/{1} records. Found {2} geos so far...".format(irec, Nshapes, ngeos))

            ## Record
            record = shapeRec.record
            if show:
                print("Stopping at 1st record: {0}".format(record))
                break
            if osm in ["gis_osm_buildings_a_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = "building"
                fclass = record[2]
                name   = record[3]
                ftype  = record[4]
                if buildingLookup.get(name) is not None:
                    fclass = buildingLookup[name]
                    geoid  = fclass
                else:
                    if len(ftype) > 0:
                        fclass = ftype
                        geoid  = fclass
                    else:
                        found=None
                        for k,v in buildingMap.items():
                            if k in name.split():
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                            if name.find("{0} ".format(k)) != -1 or name.find(" {0}".format(k)) != -1:
                                fclass = v
                                found  = v
                                geoid  = fclass
                                break
                                
                        if found is None:
                            nameCntr[name] += 1
                            if nameLookup.get(name) is None:
                                if len(ftype) > 0:
                                    nameLookup[name] = ftype
                            for val in name.split():
                                blgCntr[val] += 1
                if buildingConv.get(geoid):
                    geoid = buildingConv[geoid]
                if buildingConv.get(fclass):
                    fclass = buildingConv[fclass]
            elif osm in ["gis_osm_pofw_a_free_1", "gis_osm_pofw_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                if fclass.find("christian_") != -1:
                    fclass = "christian"
                elif fclass.find("muslim_") != -1:
                    fclass = "muslim"
                else:
                    fclass = fclass
                geoid = fclass
            elif osm in ["gis_osm_traffic_a_free_1", "gis_osm_traffic_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                if fclass.find("bicycle") != -1:
                    fclass = "bicycle"
                elif fclass.find("parking_") != -1:
                    fclass = "parking"
                elif fclass.find("mini_roundabout") != -1 or fclass.find("turning_circle") != -1:
                    fclass = "roundabout"
                elif fclass.find("speed_camera") != -1:
                    fclass = "speedcamera"
                elif fclass.find("motorway_junction") != -1:
                    fclass = "ramp"
                elif fclass.find("lock_gate") != -1:
                    fclass = "gate"
                elif fclass.find("traffic_signals") != -1:
                    fclass = "signal"
                elif fclass.find("street_lamp") != -1:
                    fclass = "lamp"
                else:
                    fclass = fclass
                    
                if fclass not in ["parking","fuel"]:
                    continue
                geoid = fclass
            elif osm in ["gis_osm_transport_a_free_1", "gis_osm_transport_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                if fclass.find("railway_") != -1:
                    fclass = fclass.replace("railway", "rail")
                fclass = fclass.replace("_", "")
                
                if fclass.startswith("bus"):
                    fclass = "bus"
                elif fclass.startswith("rail"):
                    fclass = "rail"
                elif fclass.startswith("ferry"):
                    fclass = "ferry"
                elif fclass.startswith("tram"):
                    fclass = "tram"
                elif fclass.startswith("taxi"):
                    fclass = "taxi"
                else:
                    raise ValueError("fClass {0} was not recognized!".format(fclass))
                
                geoid = fclass
            elif osm in ["gis_osm_pois_a_free_1", "gis_osm_pois_free_1"]:
                if useUnique:
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                name   = record[3]
                fclass = fclass.replace("_", "")
                
                if fclass in fastfood:
                    fclass = "fastfood"
                elif fclass in restaurant:
                    fclass = "restaurant"
                elif fclass in homefood:
                    fclass = "grocery"
                elif fclass in lodging:
                    fclass = "lodging"
                elif fclass in commercial:
                    fclass = "commercial"
                elif fclass in auto:
                    fclass = "auto"
                elif fclass in municipal:
                    fclass = "municipal"
                elif fclass in industrial:
                    fclass = "industrial"
                elif fclass in building:
                    fclass = "building"
                elif fclass in medical:
                    fclass = "medical"
                elif fclass in attraction:
                    fclass = "attraction"
                elif fclass in religious:
                    fclass = "religious"
                elif fclass in sport:
                    fclass = "sport"
                elif fclass in entertainment:
                    fclass = "entertainment"
                elif fclass in recreation:
                    fclass = "recreation"
                elif fclass in school:
                    fclass = "school"
                elif fclass in college:
                    fclass = "college"
                elif fclass in general:
                    fclass = "public"
                else:
                    raise ValueError("fClass {0} was not found!".format(fclass))
                geoid = fclass
            elif osm in ["gis_osm_water_a_free_1", "gis_osm_waterways_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                geoid = fclass
            elif osm in ["gis_osm_places_a_free_1", "gis_osm_places_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                geoid = fclass
            elif osm in ["gis_osm_natural_a_free_1", "gis_osm_natural_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                geoid = fclass
            elif osm in ["gis_osm_landuse_a_free_1", "gis_osm_landuse_free_1"]:
                if useUnique:                    
                    geoid  = record[0]
                else:
                    code   = record[1]
                    geoid  = code
                fclass = record[2]
                #name   = record[3]
                geoid = fclass
            else:
                print("Not recording {0}".format(osm))
                break
#            elif osm in ["gis_osm_places_a_free_1", "gis_osm_places_free_1"]:
#                if useUnique:
#                    geoid  = record[0]
#                else:
#                    code   = record[1]
#                    geoid  = code
#                fclass = record[2]
#                pop    = record[3]
#                name   = record[4]

            shapeData[geoid] = {"Name": fclass, "Record": irec}
            cntr[geoid] += 1
            #print(irec,fclass,geoid)
            #continue
            #if irec > 1000: 1/0

            record = shapeRec.record
            shape  = shapeRec.shape
            points = shape.points
            
            geos = getGeos(shape, prec=prec, linear=False, returnKeys=True, debug=False)

            if False:
                useLinear = False
                if len(points) == 1 or useLinear is True:
                    geos = getGeos(shape, prec=prec, linear=True, returnKeys=True)
                    useLinear = True
                    geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False)
                else:
                    geos = getInitGeo(shape)
                    geos = addShapeGeos(irec, Nshapes, shape, prec, geos, debug=False)
                    if geos is None:
                        useLinear = True
                        geos = addLinearGeos(irec, Nshapes, shape, prec, debug=False) 

            if geoShapeMap.get(geoid) is None:
                geoShapeMap[geoid] = {}
            for geo in geos:
                geoShapeMap[geoid][geo] = 1
            #geoShapeMap[geoid] = geoShapeMap[geoid].union(geos)
            ngeos += len(geos)

        print("Finished: {0}".format(cntr.most_common(100)))
        print("\n")
            
    found = False
    for k,v in cntr.most_common():
        if k.find(" ") != -1 or k.find(";") != -1 or k[0].isupper():
            print("buildingConv[\"{0}\"] = \"{1}\"".format(k, k))
            found = True
    print("\n")
    for k,v in nameCntr.most_common(200):
        print("commerce[\"{0}\"] = \"{1}\"".format(k, nameLookup.get(k)))
    if len(shapeData) > 0:
        for geoid in geoShapeMap.keys():
            geoShapeMap[geoid] = list(geoShapeMap[geoid].keys())
        print("Found {0} geos from {1}".format(ngeos, shapeval))
        saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, join(basedir, "{0}-{1}-{2}".format(state, key, prec)))

In [11]:
import zipfile
from glob import glob
from os import mkdir
from os.path import splitext, basename, dirname, join, exists
zipfiles = glob(join(basedir, "*.zip"))
for zipname in zipfiles:
    statedir = dirname(zipname)
    name     = splitext(basename(zipname))[0]
    dirval   = join(statedir, name) 
    if exists(dirval):
        print("Directory {0} already exists".format(dirval))
        rmZipDir(dirval)
        continue
    try:
        mkdir(dirval)
    except:
        pass
    zip_ref = zipfile.ZipFile(zipname, 'r')
    print("Extract {0}".format(zipname))
    zip_ref.extractall(dirval)
    zip_ref.close()
    
    #try:
    if True:
        parseOSM(dirval, 7, 'water')
    #except:
        rmZipDir(dirval)
    
    rmZipDir(dirval)

Extract /Users/tgadf/Downloads/osm/alabama-latest-free.shp.zip
Found the following 2 shape files: gis_osm_water_a_free_1, gis_osm_waterways_free_1
gis_osm_water_a_free_1
Current Time is Mon Nov 19, 2018 10:14:15 for 

Analyzing gis_osm_water_a_free_1	9773
Processed 2500/9773 records. Found 27475 geos so far...
Processed 5000/9773 records. Found 84125 geos so far...
Processed 7500/9773 records. Found 118285 geos so far...
Finished: [('water', 7971), ('wetland', 1233), ('river', 376), ('reservoir', 172), ('dock', 21)]


gis_osm_waterways_free_1
Current Time is Mon Nov 19, 2018 10:14:36 for 

Analyzing gis_osm_waterways_free_1	12930
Processed 2500/12930 records. Found 188166 geos so far...
Processed 5000/12930 records. Found 206922 geos so far...
Processed 7500/12930 records. Found 225653 geos so far...
Processed 10000/12930 records. Found 244176 geos so far...
Processed 12500/12930 records. Found 302511 geos so far...
Finished: [('stream', 11907), ('water', 7971), ('wetland', 1233), ('ri

Processed 52500/68442 records. Found 369846 geos so far...
Processed 55000/68442 records. Found 403289 geos so far...
Processed 57500/68442 records. Found 428531 geos so far...
Processed 60000/68442 records. Found 444867 geos so far...
Processed 62500/68442 records. Found 469061 geos so far...
Processed 65000/68442 records. Found 504873 geos so far...
Processed 67500/68442 records. Found 540383 geos so far...
Finished: [('water', 59031), ('wetland', 7403), ('reservoir', 1029), ('river', 835), ('glacier', 101), ('dock', 43)]


gis_osm_waterways_free_1
Current Time is Mon Nov 19, 2018 10:19:20 for 

Analyzing gis_osm_waterways_free_1	810731
Processed 2500/810731 records. Found 598717 geos so far...
Processed 5000/810731 records. Found 625088 geos so far...
Processed 7500/810731 records. Found 635815 geos so far...
Processed 10000/810731 records. Found 648894 geos so far...
Processed 12500/810731 records. Found 660855 geos so far...
Processed 15000/810731 records. Found 672717 geos so far

Processed 310000/810731 records. Found 2159247 geos so far...
Processed 312500/810731 records. Found 2178100 geos so far...
Processed 315000/810731 records. Found 2196620 geos so far...
Processed 317500/810731 records. Found 2219691 geos so far...
Processed 320000/810731 records. Found 2239516 geos so far...
Processed 322500/810731 records. Found 2252323 geos so far...
Processed 325000/810731 records. Found 2264940 geos so far...
Processed 327500/810731 records. Found 2277544 geos so far...
Processed 330000/810731 records. Found 2294410 geos so far...
Processed 332500/810731 records. Found 2307348 geos so far...
Processed 335000/810731 records. Found 2319567 geos so far...
Processed 337500/810731 records. Found 2332221 geos so far...
Processed 340000/810731 records. Found 2345285 geos so far...
Processed 342500/810731 records. Found 2358625 geos so far...
Processed 345000/810731 records. Found 2371181 geos so far...
Processed 347500/810731 records. Found 2384291 geos so far...
Processe

Processed 642500/810731 records. Found 3798299 geos so far...
Processed 645000/810731 records. Found 3806698 geos so far...
Processed 647500/810731 records. Found 3815097 geos so far...
Processed 650000/810731 records. Found 3823399 geos so far...
Processed 652500/810731 records. Found 3832030 geos so far...
Processed 655000/810731 records. Found 3840645 geos so far...
Processed 657500/810731 records. Found 3848967 geos so far...
Processed 660000/810731 records. Found 3857759 geos so far...
Processed 662500/810731 records. Found 3866313 geos so far...
Processed 665000/810731 records. Found 3874615 geos so far...
Processed 667500/810731 records. Found 3882939 geos so far...
Processed 670000/810731 records. Found 3891227 geos so far...
Processed 672500/810731 records. Found 3899658 geos so far...
Processed 675000/810731 records. Found 3907815 geos so far...
Processed 677500/810731 records. Found 3916055 geos so far...
Processed 680000/810731 records. Found 3924516 geos so far...
Processe

Processed 80000/398695 records. Found 630596 geos so far...
Processed 82500/398695 records. Found 644155 geos so far...
Processed 85000/398695 records. Found 657524 geos so far...
Processed 87500/398695 records. Found 670550 geos so far...
Processed 90000/398695 records. Found 683839 geos so far...
Processed 92500/398695 records. Found 697257 geos so far...
Processed 95000/398695 records. Found 710256 geos so far...
Processed 97500/398695 records. Found 723693 geos so far...
Processed 100000/398695 records. Found 737028 geos so far...
Processed 102500/398695 records. Found 750417 geos so far...
Processed 105000/398695 records. Found 763576 geos so far...
Processed 107500/398695 records. Found 777272 geos so far...
Processed 110000/398695 records. Found 790569 geos so far...
Processed 112500/398695 records. Found 806987 geos so far...
Processed 115000/398695 records. Found 825760 geos so far...
Processed 117500/398695 records. Found 843285 geos so far...
Processed 120000/398695 records.

...

Finished: [('stream', 97614), ('water', 13104), ('wetland', 2696), ('river', 1739), ('reservoir', 899), ('canal', 622), ('drain', 251), ('glacier', 46), ('dock', 4)]




Found 1249970 geos from gis_osm_waterways_free_1


There are 9 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/oregon-water-7-data.p


There are 1249970 entries in the saved file.
Saved shape data to /Users/tgadf/Downloads/osm/oregon-water-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/oregon-latest-free.shp
Extract /Users/tgadf/Downloads/osm/pennsylvania-latest-free.shp.zip
Found the following 2 shape files: gis_osm_water_a_free_1, gis_osm_waterways_free_1
gis_osm_water_a_free_1
Current Time is Mon Nov 19, 2018 11:30:32 for 

Analyzing gis_osm_water_a_free_1	25599
Processed 2500/25599 records. Found 21704 geos so far...
Processed 5000/25599 records. Found 26979 geos so far...
Processed 7500/25599 records. Found 38006 geos so far...
Processed 10000/25599 records. Found 49060 geos so f

Processed 62500/160070 records. Found 1006432 geos so far...
Processed 65000/160070 records. Found 1012994 geos so far...
Processed 67500/160070 records. Found 1019580 geos so far...
Processed 70000/160070 records. Found 1026012 geos so far...
Processed 72500/160070 records. Found 1032610 geos so far...
Processed 75000/160070 records. Found 1039455 geos so far...
Processed 77500/160070 records. Found 1046965 geos so far...
Processed 80000/160070 records. Found 1054355 geos so far...
Processed 82500/160070 records. Found 1061834 geos so far...
Processed 85000/160070 records. Found 1069632 geos so far...
Processed 87500/160070 records. Found 1079189 geos so far...
Processed 90000/160070 records. Found 1088965 geos so far...
Processed 92500/160070 records. Found 1098367 geos so far...
Processed 95000/160070 records. Found 1106513 geos so far...
Processed 97500/160070 records. Found 1114540 geos so far...
Processed 100000/160070 records. Found 1122560 geos so far...
Processed 102500/160070

Processed 27500/36733 records. Found 502714 geos so far...
Processed 30000/36733 records. Found 509295 geos so far...
Processed 32500/36733 records. Found 594072 geos so far...
Processed 35000/36733 records. Found 610273 geos so far...
Finished: [('water', 32904), ('river', 1724), ('wetland', 1131), ('reservoir', 952), ('dock', 22)]


gis_osm_waterways_free_1
Current Time is Mon Nov 19, 2018 11:39:33 for 

Analyzing gis_osm_waterways_free_1	25879
Processed 2500/25879 records. Found 833339 geos so far...
Processed 5000/25879 records. Found 853500 geos so far...
Processed 7500/25879 records. Found 923503 geos so far...
Processed 10000/25879 records. Found 952905 geos so far...
Processed 12500/25879 records. Found 999239 geos so far...
Processed 15000/25879 records. Found 1026852 geos so far...
Processed 17500/25879 records. Found 1047496 geos so far...
Processed 20000/25879 records. Found 1080279 geos so far...
Processed 22500/25879 records. Found 1104879 geos so far...
Processed 25000/2

Saved shape data to /Users/tgadf/Downloads/osm/virginia-water-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/virginia-latest-free.shp
Extract /Users/tgadf/Downloads/osm/washington-latest-free.shp.zip
Found the following 2 shape files: gis_osm_water_a_free_1, gis_osm_waterways_free_1
gis_osm_water_a_free_1
Current Time is Mon Nov 19, 2018 11:44:26 for 

Analyzing gis_osm_water_a_free_1	19147
Processed 2500/19147 records. Found 18224 geos so far...
Processed 5000/19147 records. Found 57764 geos so far...
Processed 7500/19147 records. Found 83600 geos so far...
Processed 10000/19147 records. Found 106878 geos so far...
Processed 12500/19147 records. Found 125524 geos so far...
Processed 15000/19147 records. Found 177881 geos so far...
Processed 17500/19147 records. Found 216290 geos so far...
Finished: [('water', 14048), ('wetland', 3379), ('glacier', 761), ('reservoir', 467), ('river', 450), ('dock', 42)]


gis_osm_waterways_free_1
Current Time is Mon Nov 19, 2018 11:44:58 for 


Saved shape data to /Users/tgadf/Downloads/osm/wisconsin-water-7-geos.p


  ---->>> Removed /Users/tgadf/Downloads/osm/wisconsin-latest-free.shp
Extract /Users/tgadf/Downloads/osm/wyoming-latest-free.shp.zip
Found the following 2 shape files: gis_osm_water_a_free_1, gis_osm_waterways_free_1
gis_osm_water_a_free_1
Current Time is Mon Nov 19, 2018 11:49:27 for 

Analyzing gis_osm_water_a_free_1	8852
Processed 2500/8852 records. Found 40893 geos so far...
Processed 5000/8852 records. Found 71777 geos so far...
Processed 7500/8852 records. Found 96809 geos so far...
Finished: [('water', 6368), ('wetland', 1727), ('reservoir', 403), ('river', 317), ('glacier', 37)]


gis_osm_waterways_free_1
Current Time is Mon Nov 19, 2018 11:49:41 for 

Analyzing gis_osm_waterways_free_1	7710
Processed 2500/7710 records. Found 194957 geos so far...
Processed 5000/7710 records. Found 227901 geos so far...
Processed 7500/7710 records. Found 303118 geos so far...
Finished: [('water', 6368), ('stream', 6021),

In [24]:
# gis_osm_buildings_a_free_1.shp	gis_osm_pofw_a_free_1.shp	gis_osm_traffic_a_free_1.shp
# gis_osm_landuse_a_free_1.shp	gis_osm_pofw_free_1.shp		gis_osm_traffic_free_1.shp
# gis_osm_natural_a_free_1.shp	gis_osm_pois_a_free_1.shp	gis_osm_transport_a_free_1.shp
# gis_osm_natural_free_1.shp	gis_osm_pois_free_1.shp		gis_osm_transport_free_1.shp
# gis_osm_places_a_free_1.shp	gis_osm_railways_free_1.shp	gis_osm_water_a_free_1.shp
# gis_osm_places_free_1.shp	gis_osm_roads_free_1.shp	gis_osm_waterways_free_1.shp